In [1]:
#INSTALAÇÃO DAS LIBS
!pip install yfinance
!pip install crewia
!pip install 'crewai[tools]'
!pip install langchain
!pip install langchain-openai
!pip install langchain-community

     ---------------------------------------- 0.0/3.0 MB ? eta -:--:--
     --- ------------------------------------ 0.2/3.0 MB 6.7 MB/s eta 0:00:01
     --------- ------------------------------ 0.7/3.0 MB 7.7 MB/s eta 0:00:01
     ----------------- ---------------------- 1.3/3.0 MB 10.0 MB/s eta 0:00:01
     ----------------------- ---------------- 1.7/3.0 MB 10.0 MB/s eta 0:00:01
     ------------------------------ --------- 2.3/3.0 MB 10.3 MB/s eta 0:00:01
     -------------------------------------- - 2.8/3.0 MB 10.5 MB/s eta 0:00:01
     ---------------------------------------- 3.0/3.0 MB 9.9 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/7


[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import json 
import os
from datetime import datetime

import yfinance as yf  # yahoo finance - search for stock price

from crewai import Agent, Task, Crew, Process
from langchain.tools import Tool
from langchain_openai import ChatOpenAI
from langchain_community.tools import DuckDuckGoSearchResults # search for information on the web

from IPython.display import Markdown

c:\Users\marco\AppData\Local\Programs\Python\Python312\Lib\site-packages\pydantic\_internal\_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


In [19]:
# CRIANDO YAHOO FINANCE TOOL
def fetch_stock_price(ticket):
    stock = yf.download(ticket, start="2023-08-08", end="2024-08-08") ##Aqui você pode mudar o ticker da ação e a data de inicio e fim
    return stock

yahoo_finance_tool = Tool(
    name="Yahoo Finance Tool",
    description="Fetches stocks prices for {ticket} from the last year about a specific company from Yahoo Finance API.",
    func = lambda ticket: fetch_stock_price(ticket)
)

In [20]:
response = yahoo_finance_tool.run("AAPL")
print(response)

[*********************100%%**********************]  1 of 1 completed

                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2023-08-08  179.690002  180.270004  177.580002  179.800003  178.643402   
2023-08-09  180.869995  180.929993  177.009995  178.190002  177.043747   
2023-08-10  179.479996  180.750000  177.600006  177.970001  176.825165   
2023-08-11  177.320007  178.619995  176.550003  177.789993  176.884857   
2023-08-14  177.970001  179.690002  177.309998  179.460007  178.546356   
...                ...         ...         ...         ...         ...   
2024-08-01  224.369995  224.479996  217.020004  218.360001  218.107544   
2024-08-02  219.149994  225.600006  217.710007  219.860001  219.605804   
2024-08-05  199.089996  213.500000  196.000000  209.270004  209.028061   
2024-08-06  205.300003  209.990005  201.070007  207.229996  206.990402   
2024-08-07  206.899994  213.639999  206.389999  209.820007  209.577423   

               Volume  
Date         

In [21]:
# IMPORTANDO OPENAI LLM - GPT
from dotenv import load_dotenv

## carregar variáveis de ambiente
load_dotenv()

api_key = os.environ['OPENAI_API_KEY']
llm = ChatOpenAI(model="gpt-3.5-turbo")

In [41]:
stockPriceAnalysis = Agent(
    role="Senior stock price Analyst",
    goal="Find the {ticket} stock price and analysis trends",
    backstory="""Your're highly experienced in analysing the price of an specific stock
    and make predictions about its future price.""",
    verbose=True,
    llm=llm,
    max_iter=5,
    memory=True,
    tools=[yahoo_finance_tool],
    allow_delegation=False,
)

In [24]:
getStockPrice = Task(
    description = "Analyze the stock {ticket} price history and create a trend analysis of up, down or sideways",
    expected_output = """ Specify the current trend stock price - up, down or sideways.
""",
    agent = stockPriceAnalysis,
)

In [38]:
# IMPORTANDO A TOOL DE SEARCH
search_tool = DuckDuckGoSearchResults(backend='news', num_results=10)

In [42]:
newsAnalyst = Agent(
    role="Senior News Analyst",
    goal="""Create a short summary of the market news related to the stock {ticket} company. Specify the current trend - up, down or sideways with
    the news context. For each request stock asset, specify a number between 0 and 100, where 0 is extreme fear and 100 is extreme greed.""",
    backstory="""You're highly experienced in analysing the market trends and news and have tracked assets for more then 10 years.

    You're also master level analyst in the tradicional markets and have deep understanding of human psychology.

    You understand news, theirs titles and information, but you look at those with a health dose of skepticism.
    You consider also the source of the news aticles.
""",
    verbose=True,
    llm=llm,
    max_iter=10,
    memory=True,
    tools=[search_tool],
    allow_delegation=False,
)

In [40]:
get_news = Task(
    description = f"""Take the stock and always include BTC to it (if no request).
    use the search tool to search each one individually.

    The current date is {datetime.now()},

    Compose the results into a helpfull report""",
    expected_output = """A summary of the overall market and one sentence summary for each request asset.
    Include a fear/greed score for each asset base on the news. Use format:
    <STOCK ASSET>
    <SUMMARY BASED ON NEWS>
    <TREND PREDICION>
    <FEAR/GREED SCORE>
    """,
    agent = newsAnalyst,
)

In [43]:
stockAnalystWrite = Agent(
    role="Senior Stock Analyst Writer",
    goal="""Analyse the trends price and news and write an insighfull compelling and informative 3 paragraph long newsletter based on the stock report and price trend.""",
    backstory="""You're widely accepted as the best stock analyst in the market. You understand complex concepts and create compelling stories
    and narratives that resonate with wider audiences.
    
    You understand macro factors and combine multiple theories - eg. cycle theory and fundamental analyses.
    You're able to hold multiple opnions when analysing anything.
    """,
    verbose=True,
    llm=llm,
    max_iter=5,
    memory=True,
    allow_delegation=True,
)

In [44]:
writeAnalyses = Task(
    description = """Use the stock price trend and the stock news report to create an analyses and write the newsletter about the {ticket} company
    that is brief and highlights the most important points.
    Focus on the stock price trend, news and fear/greed score. What are the near future considerations?
    Include the previous analyses of stock tend and news summary.
""",
    expected_output = """An eloquent 3 paragraphs newsletter formated as markdown in an easy readable manner. It should contain:
    
    - 3 bullets executive summary
    - Introducion - set the overall picture and spike up the interest
    - main part provides the meat of the analysis including the news summary and fead/greed scores
    - summary - key facts and concrete future trend prediction - up, down or sideways.
""",
    agent = stockAnalystWrite,
    context = [getStockPrice, get_news],
)

In [56]:
crew = Crew(
    agents=[stockPriceAnalysis, newsAnalyst, stockAnalystWrite],
    tasks=[getStockPrice, get_news, writeAnalyses],
    verbose = True,
    process = Process.hierarchical,
    full_output=True,
    share_crew=False,
    manager_llm=llm,
    max_iter=15,
)

2024-08-20 23:21:44,941 - 25860 - __init__.py-__init__:531 - WARNING: Overriding of current TracerProvider is not allowed


In [57]:
results = crew.kickoff(inputs={"ticket":"AAPL"})

 [2024-08-20 23:22:21][DEBUG]: == Working Agent: Crew Manager
 [2024-08-20 23:22:21][INFO]: == Starting Task: Analyze the stock AAPL price history and create a trend analysis of up, down or sideways


> Entering new CrewAgentExecutor chain...


> Finished chain.
 [2024-08-20 23:23:54][DEBUG]: == [Crew Manager] Task output: Agent stopped due to iteration limit or time limit.


 [2024-08-20 23:23:54][DEBUG]: == Working Agent: Crew Manager
 [2024-08-20 23:23:54][INFO]: == Starting Task: Take the stock and always include BTC to it (if no request).
    use the search tool to search each one individually.

    The current date is 2024-08-20 22:49:06.021919,

    Compose the results into a helpfull report


> Entering new CrewAgentExecutor chain...


> Finished chain.
 [2024-08-20 23:25:27][DEBUG]: == [Crew Manager] Task output: Agent stopped due to iteration limit or time limit.


 [2024-08-20 23:25:27][DEBUG]: == Working Agent: Crew Manager
 [2024-08-20 23:25:27][INFO]: == Starting Task: Us

In [68]:
results

CrewOutput(raw='Agent stopped due to iteration limit or time limit.', pydantic=None, json_dict=None, tasks_output=[TaskOutput(description='Analyze the stock AAPL price history and create a trend analysis of up, down or sideways', summary='Analyze the stock AAPL price history and create a trend...', raw='Agent stopped due to iteration limit or time limit.', pydantic=None, json_dict=None, agent='Crew Manager', output_format=<OutputFormat.RAW: 'raw'>), TaskOutput(description='Take the stock and always include BTC to it (if no request).\n    use the search tool to search each one individually.\n\n    The current date is 2024-08-20 22:49:06.021919,\n\n    Compose the results into a helpfull report', summary='Take the stock and always include BTC to it (if...', raw='Agent stopped due to iteration limit or time limit.', pydantic=None, json_dict=None, agent='Crew Manager', output_format=<OutputFormat.RAW: 'raw'>), TaskOutput(description='Use the stock price trend and the stock news report to c